##**5.7 - Limpeza dos dados**

In [0]:
#!pip install -U pandasql
import pandas as pd
import pandasql

###Chamando CSV

In [0]:
df = pd.read_csv('ovni_data.csv')

#Excluindo coluna Indesejada
del df['Unnamed: 0']

### Remover campos em Branco, Unknown e None

In [13]:
eliminar_campos = """
   SELECT * 
   FROM df 
   WHERE lower(City) not in ('unknown', 'none', ' ') and 
   lower(State) not in ('unknown', 'none', ' ') and 
   lower(Shape) not in ('unknown', 'none', ' ')
 """
lista_result_eliminacao = pandasql.sqldf(eliminar_campos)

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,9/30/97 22:00,Madison,WI,Light,5 minutes,Strange light inside Lake Monona,3/2/04
1,9/30/97 20:00,Nova Scotia (Canada),NS,Light,8-10 seconds.,"Single light resembling a star, but moving spu...",10/30/06
2,9/28/97 23:15,San Francisco,CA,Triangle,12-15s,flying-wing shape outlined by 12-14 lights. Ap...,7/5/99
3,9/27/97 23:00,Egan,SD,Other,30 minutes,The Weirdest Thing I Have Ever Seen,2/22/05
4,9/27/97 05:00,Crestwood,KY,Disk,15 minutes,A big disk with red and green lights on the ri...,8/5/01
...,...,...,...,...,...,...,...
84024,8/1/17 14:00,Joliet,IL,Other,2 minutes,The White Cube UFO,7/25/19
84025,8/1/17 06:15,Columbus (North),GA,Fireball,3 seconds,Green streak growing in size moving from west ...,8/4/17
84026,8/1/17 02:45,Corcoran,MN,Light,Still going,Small light south west of Minneapolis maneuver...,8/4/17
84027,8/1/17 02:00,Moreno Valley,CA,Other,10 seconds,I was looking out the front windshield and loo...,8/4/17


### Manter registros que pertencem aos 51 estados dos Estados Unidos

In [0]:
estados = pd.read_csv('states.csv', sep=';')

In [14]:
consulta_estados_validos = '''
  SELECT lista_result_eliminacao.* 
  FROM lista_result_eliminacao, estados
  WHERE lista_result_eliminacao.State = estados.Abbreviation
'''

retorno_filtro_usa = pandasql.sqldf(consulta_estados_validos)

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,9/30/97 22:00,Madison,WI,Light,5 minutes,Strange light inside Lake Monona,3/2/04
1,9/28/97 23:15,San Francisco,CA,Triangle,12-15s,flying-wing shape outlined by 12-14 lights. Ap...,7/5/99
2,9/27/97 23:00,Egan,SD,Other,30 minutes,The Weirdest Thing I Have Ever Seen,2/22/05
3,9/27/97 05:00,Crestwood,KY,Disk,15 minutes,A big disk with red and green lights on the ri...,8/5/01
4,9/25/97 22:00,Clearfield,UT,Triangle,60-90 seconds,We observed a low flying craft (aprox.100yards...,1/28/99
...,...,...,...,...,...,...,...
80154,8/1/17 14:00,Joliet,IL,Other,2 minutes,The White Cube UFO,7/25/19
80155,8/1/17 06:15,Columbus (North),GA,Fireball,3 seconds,Green streak growing in size moving from west ...,8/4/17
80156,8/1/17 02:45,Corcoran,MN,Light,Still going,Small light south west of Minneapolis maneuver...,8/4/17
80157,8/1/17 02:00,Moreno Valley,CA,Other,10 seconds,I was looking out the front windshield and loo...,8/4/17


### Excluir colunas - Duration, Summary e Posted

In [15]:
excluir_col = retorno_filtro_usa.drop(['Duration', 'Summary', 'Posted'], axis=1)

,Date / Time,City,State,Shape
0,9/30/97 22:00,Madison,WI,Light
1,9/28/97 23:15,San Francisco,CA,Triangle
2,9/27/97 23:00,Egan,SD,Other
3,9/27/97 05:00,Crestwood,KY,Disk
4,9/25/97 22:00,Clearfield,UT,Triangle
...,...,...,...,...
80154,8/1/17 14:00,Joliet,IL,Other
80155,8/1/17 06:15,Columbus (North),GA,Fireball
80156,8/1/17 02:45,Corcoran,MN,Light
80157,8/1/17 02:00,Moreno Valley,CA,Other


### Mostrar registros do dataframe, no qual, os registros sejam superior à 1000 ocorrências

In [16]:
ocorrencias_shape = '''
  SELECT Shape, COUNT(Shape) as Ocorrencia 
  FROM excluir_col  
  GROUP BY Shape 
  HAVING Ocorrencia > 1000 
'''

filtro_registro = pandasql.sqldf(ocorrencias_shape)

,Shape,Ocorrencia
0,Changing,2275
1,Chevron,1041
2,Cigar,1896
3,Circle,9313
4,Cylinder,1367
5,Diamond,1313
6,Disk,4299
7,Fireball,7535
8,Flash,1684
9,Formation,2863


In [17]:
filtro_final = '''
  SELECT excluir_col.* 
  FROM excluir_col, filtro_registro
  WHERE filtro_registro.Shape = excluir_col.Shape
'''

df_final = pandasql.sqldf(filtro_final)

df_final.to_csv('df_OVNI_limpo.csv')

,Date / Time,City,State,Shape
0,9/30/97 22:00,Madison,WI,Light
1,9/28/97 23:15,San Francisco,CA,Triangle
2,9/27/97 23:00,Egan,SD,Other
3,9/27/97 05:00,Crestwood,KY,Disk
4,9/25/97 22:00,Clearfield,UT,Triangle
...,...,...,...,...
77897,8/1/17 14:00,Joliet,IL,Other
77898,8/1/17 06:15,Columbus (North),GA,Fireball
77899,8/1/17 02:45,Corcoran,MN,Light
77900,8/1/17 02:00,Moreno Valley,CA,Other
